https://www.jstor.org/stable/2329112

In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(42)

# Random data

In [116]:
stock_n = 10 # 개별 주식 수
dates = pd.date_range(start='1963-07-01', end='1990-12-31', freq='ME')
returns = pd.DataFrame(
    {f"Stock{i}": np.random.uniform(-0.05, 0.05, dates.shape[0]) 
    for i in range(stock_n)})
print("The number of dates:", dates.shape)
print("Sample of dates:", dates[:3])
print("Sample of returns:")
returns.iloc[0], sum(returns.iloc[0])

The number of dates: (330,)
Sample of dates: DatetimeIndex(['1963-07-31', '1963-08-31', '1963-09-30'], dtype='datetime64[ns]', freq='ME')
Sample of returns:


(Stock0   -0.045765
 Stock1    0.015384
 Stock2   -0.009275
 Stock3    0.016755
 Stock4   -0.024138
 Stock5   -0.001668
 Stock6    0.047822
 Stock7   -0.017585
 Stock8    0.021517
 Stock9   -0.010576
 Name: 0, dtype: float64,
 -0.007528816825977766)

In [117]:
portfolio_returns = returns.sum(axis=1)
portfolio_returns

0     -0.007529
1     -0.142868
2     -0.039662
3     -0.006617
4      0.181774
         ...   
325   -0.093531
326    0.005799
327   -0.066319
328    0.105494
329    0.156561
Length: 330, dtype: float64

In [118]:
portfolio_rolled_returns = None

In [119]:
n = dates.shape[0] 
market_returns_t = np.random.normal(loc=0.02, scale=0.05, size=n) 

market_data = pd.DataFrame({
    'Market_Returns_t_minus_1': pd.Series(market_returns_t).shift(1), 
    'Market_Returns_t': market_returns_t
})

market_data.head()

,Market_Returns_t_minus_1,Market_Returns_t
0,NaN,0.076221
1,0.076221,0.080649
2,0.080649,-0.000711
3,-0.000711,-0.027238
4,-0.027238,0.017414


# Beta


## Pre-ranking beta
개별주식의 pre-ranking 베타는 개별주식의 이전 60개월 중 이용 가능한 시기의 월수익률을 사용해서 market model을 통해 만들어진다. market model에서 시장 당월 수익률 변수의 기울기와 전월 수익률 변수의 기울기의 합을 pre-ranking 베타값으로 사용한다

In [120]:
# skip nan in market data
months = 60
nans = market_data.isna().sum().sum()
X = market_data.dropna()


In [127]:
df = {}
for s in range(stock_n):
    y = returns[f"Stock{s}"].loc[returns.index[nans:]]
    stock_beta = []
    for i in range(returns.shape[0]-nans):
        X_temp = X.iloc[i:months+i]
        y_temp = y.iloc[i:months+i]

        X_temp = sm.add_constant(X_temp)
        model = sm.OLS(y_temp, X_temp).fit()
        beta_market = model.params["Market_Returns_t"]
        beta_market_lag = model.params["Market_Returns_t_minus_1"]

        pre_ranking_beta = beta_market + beta_market_lag
        # pre_ranking_beta
        # print(dates[i+nans], pre_ranking_beta)
        stock_beta.append(pre_ranking_beta)
    df[f"Stock{s}"] = stock_beta
df = pd.DataFrame(df, index=dates[nans:])
print(df.shape)
df.head()

(329, 10)


,Stock0,Stock1,Stock2,Stock3,Stock4,Stock5,Stock6,Stock7,Stock8,Stock9
1963-08-31,0.031792,-0.032740,-0.035392,0.020102,-0.130733,0.177616,0.060755,0.054966,-0.066162,-0.028655
1963-09-30,0.045710,-0.024432,0.005457,0.034437,-0.091571,0.160883,0.093792,0.065117,-0.051950,-0.026022
1963-10-31,0.038498,-0.013478,0.020764,0.041785,-0.103315,0.151701,0.094649,0.059197,-0.041342,-0.029671
1963-11-30,0.023726,0.009171,0.004356,0.041138,-0.086019,0.165261,0.073090,0.048708,-0.044455,-0.022803
1963-12-31,0.028989,0.020800,0.015433,0.036675,-0.085912,0.170298,0.084563,0.056928,-0.031144,-0.025216


## Beta for Portfolio with full period

$r_{p,t} = \beta _0 + \beta _ 1 \cdot r_{m, t-1} + \beta _2 \cdot_{m,t} + \varepsilon _p$

In [133]:
X = market_data[['Market_Returns_t_minus_1', 'Market_Returns_t']][nans:]
y = portfolio_returns[nans:]

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.params)

beta_0 = model.params['const']  # 절편
beta_1 = model.params['Market_Returns_t_minus_1']  # 시장 전월 수익률 변수의 기울기
beta_2 = model.params['Market_Returns_t']  # 시장 당월 수익률 변수의 기울기

# 합산 베타: 시장 수익률이 개별 포트폴리오 수익률에 
# 영향을 끼칠 때 시차가 존재할 수 있음을 감안하기 위해서 사용
pre_ranking_beta = beta_1 + beta_2

print("Pre-ranking beta 값:", pre_ranking_beta)


const                       0.005987
Market_Returns_t_minus_1   -0.091214
Market_Returns_t            0.035878
dtype: float64
Pre-ranking beta 값: -0.05533599789036576


# Fama MacBeth Regression

In [1]:


np.random.seed(0)
n_periods = 100
n_cross_section = 10
n_obs = n_periods * n_cross_section
data = pd.DataFrame({
    'Time': np.repeat(range(n_periods), n_cross_section),
    'Cross_Section': np.tile(range(n_cross_section), n_periods),
    'X1': np.random.randn(n_obs),
    'X2': np.random.randn(n_obs),
    'X3': np.random.randn(n_obs),
    'Y': np.random.randn(n_obs),
})

data.sort_values(by=['Cross_Section', 'Time'], inplace=True)
print(data.shape)
data.head()

(1000, 6)


,Time,Cross_Section,X1,X2,X3,Y
0,0,0,1.764052,0.555963,-1.532921,1.593274
10,1,0,0.144044,-1.000330,0.371173,0.279196
20,2,0,-2.552990,1.015665,-1.913743,-2.810668
30,3,0,0.154947,-0.753704,0.743554,0.966306
40,4,0,-1.048553,1.669251,-1.216077,-0.067945


In [2]:
cross_data = []

for cross_section, df in data.groupby('Cross_Section'):
    x = sm.add_constant(df[['X1', 'X2', 'X3']])
    y = df['Y'] 
    model = sm.OLS(y, x)
    result = model.fit()
    cross_data.append(result.params)

cross_data = pd.DataFrame(cross_data)
print(cross_data.shape)
cross_data.head()

(10, 4)


,const,X1,X2,X3
0,-0.060058,0.203161,0.049822,0.105252
1,0.165471,-0.163630,0.053271,-0.108840
2,-0.128742,-0.031212,0.038470,-0.080620
3,-0.117634,0.093704,0.244994,0.083918
4,-0.086229,-0.070633,-0.090680,-0.037857


In [3]:
# The regressors in every regression are the same collection of i. 
# Only the dependent variable changes from one regression to the other.

time_data = []
x = cross_data

for time, df in data.groupby('Time'):
    y = df['Y'].values

    model = sm.OLS(y, x)
    result = model.fit()
    time_data.append(result.params)

time_data = pd.DataFrame(time_data)
print(time_data.shape)
time_data.head()

(100, 4)


,const,X1,X2,X3
0,4.091892,3.229014,2.053984,2.874924
1,0.969862,0.419557,-1.837971,1.003579
2,3.033017,-4.763284,-2.797892,0.882572
3,1.423884,5.716954,-3.946337,-3.697416
4,-0.321093,-4.794623,-0.898722,4.788780


In [4]:

print("Mean of betas", time_data.mean(axis=0), sep="\n")
print()
print("Standard deviation of betas", time_data.std(axis=0), sep="\n")

Mean of betas
const    1.038324
X1      -0.041370
X2      -0.040311
X3      -0.100653
dtype: float64

Standard deviation of betas
const    2.875527
X1       2.927050
X2       2.954771
X3       4.133308
dtype: float64
